In [2]:
!pip install -qU \
    langchain==0.0.292 \
    openai==0.28.0 \
    datasets==2.10.1 \
    pinecone-client==2.2.4 \
    tiktoken==0.5.1 \
    datasets-2.16.0

ERROR: Could not find a version that satisfies the requirement datasets-2.16.0 (from versions: none)
ERROR: No matching distribution found for datasets-2.16.0

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import os

In [4]:
def getkey(keyfile, dir_keys):
    try:
        with open(os.path.join(dir_keys, keyfile + '.key'), 'r') as f:
            lines = f.readlines()
            for line in lines:
                if len(line.strip()) > 0:
                    envvar, value = line.split(':')
                    value = value.strip()
                    os.environ[envvar] = value
    except:
        os.environ[envvar] = ""

In [5]:
home = os.environ["HOME"]
dir_keys = home + '/keys'
os.makedirs(dir_keys, exist_ok=True)
getkey("openai", dir_keys)
    

In [6]:
getkey("pinecone", dir_keys)

In [7]:
from langchain.chat_models import ChatOpenAI

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or "YOUR_API_KEY"

chat = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    #model='gpt-4'
    #model='gpt-4-1106-preview'
    model='gpt-4o'
)

In [8]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

In [9]:
therastream_SM = SystemMessage(content="""
You are an expert in psychotherapeutic methods, with a particular emphasis on the twelve steps program, diagnosis and understanding of codependency and the role of the inner child, tools such as meditation and yoga / yoga nidra and mindfullness, and "Awareness, Acceptance, Action" methodology to support inner peace, and with communication methods including non-violent communication and the communications model from Friedemann Schultz von Thun,  .
You are very familiar with the scientific literature in these topics, self-help literature and Buddhism.
You are also an expert with python coding and the generation of simple and user-friendly applications using the streamlit framework.
You are an expert in both English and German languages.
""")
therastream_HM = HumanMessage(content="""
I want to create a simple and effective streamlit application for my therapist called "Therastream".
She is looking to write a self-help book based on all her experience. 
The tool should support this goal with the following streamlit elements:
1. Connect to OpenAI: provide API key if not already given and connect to chat.
2. a. Ask an AI Therapist: Ask the AI a question. This adds the question as a prompt to an appropriately crafted system prompt which seeks to maximize the usefullness and appropriateness of style of the answer.
2. b. Follow up questions to the AI Therapist ( via a conversation interface, with memory of previous queries and responses ).
3. a. Create Book Content: Ask an AI self-help expert to provide a summary of a particular topic.
3. b. Follow up feedback: The user provides feedback on the suggested book content, and the AI self-help expert updates its output accordingly.
4. Save content : Save the conversation to disk.
5. Load content : Load a conversation from disk.
6. Reset content

You should provide the python code to create this application.
""")
therastream_messages = [therastream_SM, therastream_HM]
therastream_res = chat(therastream_messages)
print (therastream_res.content)

Creating a Streamlit application like "Therastream" involves several steps. Below is a simplified version of the Python code to create this application. This app will utilize OpenAI's API to interact with AI models, so ensure you have access to an API key.

Ensure you have the necessary libraries installed:

```bash
pip install streamlit openai
```

Here's a basic outline of the code:

```python
import streamlit as st
import openai
import json

# Function to initialize OpenAI API
def initialize_openai(api_key):
    openai.api_key = api_key

# Function to call OpenAI API
def ask_openai(prompt):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=150
    )
    return response.choices[0].text.strip()

# Streamlit application
def main():
    st.title("Therastream")

    # 1. API Key
    api_key = st.text_input("Enter OpenAI API Key", type="password")
    if api_key:
        initialize_openai(api_key)
        st.success("Conn

In [10]:
therastream_messages_2 = therastream_messages.copy()
therastream_messages_2.append(therastream_res)

In [14]:
therastream_messages_2.append(HumanMessage(content="""
Nice. I need 2 changes.

1. The code you provided is out of date. The OpenAI interface no longer supports openai.Completion. 
Please rewrite to use the langchain.schema interface as follows:

from langchain.chat_models import ChatOpenAI
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or "YOUR_API_KEY"
chat = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model='gpt-4o'
)
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)
# initial message:
messages = [ SystemMessage(content=system_prompt), HumanMessage(content=prompt) ]
aimessage = chat(messages)
messages.append(aimessage)
# on continuation of the conversation:
messages.append(HumanMessage(content=new_prompt))
aimessage = chat(messages)
messages.append(aimessage)
# etc

2. I need an additional free text box to provide additional instructions as to what the therapist needs to be particular expert in. 
The first call to the LLM should take this input and use it to build an appropriate system prompt for the therapist and the therapy expert.

"""))
therastream_res_2 = chat(therastream_messages_2)
print (therastream_res_2.content)                      
                                        

Certainly! Let's update the Streamlit application to use the `langchain` library's `ChatOpenAI` interface and add a free text box for specifying the therapist's expertise. Here's the updated code:

```python
import streamlit as st
import os
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

# Function to initialize OpenAI API
def initialize_openai(api_key):
    os.environ["OPENAI_API_KEY"] = api_key

# Function to create a chat instance
def create_chat_instance(api_key):
    return ChatOpenAI(
        openai_api_key=api_key,
        model='gpt-4o'
    )

# Streamlit application
def main():
    st.title("Therastream")

    # 1. API Key
    api_key = st.text_input("Enter OpenAI API Key", type="password")
    if api_key:
        initialize_openai(api_key)
        chat = create_chat_instance(api_key)
        st.success("Connected to OpenAI!")

    # Initialize session state for conversation and system prompt
    if 'conversation' not in s

In [25]:
therastream_SM_v2 = SystemMessage(content="""
You are an expert in psychotherapeutic methods, with a particular emphasis on the twelve steps program, diagnosis and understanding of codependency and the role of the inner child, tools such as meditation and yoga / yoga nidra and mindfullness, and "Awareness, Acceptance, Action" methodology to support inner peace, and with communication methods including non-violent communication and the communications model from Friedemann Schultz von Thun,  .
You are very familiar with the scientific literature in these topics, self-help literature and Buddhism.
You are also an expert with python coding and the generation of simple and user-friendly applications using the streamlit framework.
You are an expert in both English and German languages.
""")
with open('/Users/david.wilman/Projects/LLMs/repos/RAG_Experiment/Notebooks/therastream/pages/3_therapist_assistant.py', 'r') as file:
    code = file.read()
therastream_HM_v2 = HumanMessage(content=f"""
I have a built a simple streamlit application, named "Therastream", a chatbot therapist.
One of the pages for this application should act as a chatbot to ask therapy questions, with an inbuilt system prompt provided elsewhere.

I would like the following adaptations and additions:

1. Currently, the Follow-up Conversation option does not work because it cannot access the local variable "Messages". 
Please provide an integrated approach to directly use the global streamlit variable st.session_state.conversation["therapist"].

2. I would like an integrated UI for the whole conversation rather than separate interfaces for the first question and any follow-up question. Otherwise intermediate questions and answers are lost from the UI.

<Code>: {code}
""")
therastream_messages_v2 = [therastream_SM_v2, therastream_HM_v2]


In [27]:
therastream_res_v2 = chat(therastream_messages_v2)
print (therastream_res_v2.content)    

To address your requirements, let's integrate the changes to ensure that the `Follow-up Conversation` option works correctly and the conversation UI is unified. Here's the modified code:

```python
import streamlit as st
import json
from langchain.schema import SystemMessage, HumanMessage

# Initialize session state variables if they don't exist
if "chat" not in st.session_state:
    st.session_state.chat = None

if "conversation" not in st.session_state:
    st.session_state.conversation = {"therapist": []}

chat = st.session_state.chat

if chat is None:
    st.write("Goto Connect tab to access functionality.")
else:

    # Unified conversation interface
    st.header("Therapist Conversation")

    # Display existing conversation
    if st.session_state.conversation["therapist"]:
        for message in st.session_state.conversation["therapist"]:
            if message["role"] == "user":
                st.write(f"You: {message['content']}")
            elif message["role"] == "assista

In [30]:
therastream_messages_v2_2 = therastream_messages_v2
therastream_messages_v2_2.append(therastream_res_v2)
therastream_messages_v2_2.append(HumanMessage(content="""
Update the code so that the save and load conversation options use a dialogue box with access to the file system, to more easily select a file to load or folder destination.
Also create an additional reset button to reset the conversation, which should be confirmed by the user.
"""))
therastream_res_v2_2 = chat(therastream_messages_v2_2)
print (therastream_res_v2_2.content)    

To incorporate file selection dialogues for saving and loading conversations, as well as a reset button with user confirmation, we can make use of the `streamlit.file_uploader` and `streamlit.download_button` features. Below is the updated code:

```python
import streamlit as st
import json
from langchain.schema import SystemMessage, HumanMessage
import os

# Initialize session state variables if they don't exist
if "chat" not in st.session_state:
    st.session_state.chat = None

if "conversation" not in st.session_state:
    st.session_state.conversation = {"therapist": []}

chat = st.session_state.chat

if chat is None:
    st.write("Goto Connect tab to access functionality.")
else:

    # Unified conversation interface
    st.header("Therapist Conversation")

    # Display existing conversation
    if st.session_state.conversation["therapist"]:
        for message in st.session_state.conversation["therapist"]:
            if message["role"] == "user":
                st.write(f"You

In [11]:
import os
from pdb import set_trace

def read_python_files(main_file, subdirectories):
    """
    Reads the contents of the main application file and all .py files within provided subdirectories.
    Prepend each file's content with its filename and append a marker.

    Args:
        main_file (str): The path to the main application file.
        subdirectories (list): A list of subdirectory paths to search for .py files.

    Returns:
        str: A single string containing the contents of all the specified .py files.
    """
    # Function to read a file and format its content
    def read_file(file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        return f"<{os.path.basename(file_path)}>\n\n{content}\n\n"

    # Initialize an empty string to hold all file contents
    all_contents = ""

    # Read the main application file
    if os.path.isfile(main_file):
        all_contents += read_file(main_file)

    # Iterate over each subdirectory
    for subdir in subdirectories:
        path = os.path.join(os.path.dirname(main_file),subdir)
        if os.path.isdir(path):
            # List all .py files in the subdirectory
            for root, _, files in os.walk(path):
                for file in files:
                    if file.endswith('.py') and 'checkpoint' not in file:
                        file_path = os.path.join(root, file)
                        all_contents += read_file(file_path)

    return all_contents

In [12]:
import requests
from bs4 import BeautifulSoup

def scrape_github_discussion_markdown(url):
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code != 200:
        raise Exception(f"Failed to retrieve content from {url}")
    
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the element that contains the markdown content
    # The markdown content in GitHub discussions is usually within a <pre><code> block
    markdown_blocks = soup.find_all('pre')
    
    # Extract text from each markdown block and join them
    markdown_content = '\n\n'.join(block.get_text() for block in markdown_blocks)
    
    return markdown_content

# Usage example
url = 'https://github.com/openai/openai-python/discussions/742#discussion-5829467'
markdown_content = scrape_github_discussion_markdown(url)
print(markdown_content)

pip install --upgrade openai


openai migrate


curl -fsSL https://docs.grit.io/install | bash
grit install
grit apply openai

# old
import openai

openai.api_key = os.environ['OPENAI_API_KEY']

# new
from openai import OpenAI

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

# before
import json
import openai

completion = openai.Completion.create(model='curie')
print(completion['choices'][0]['text'])
print(completion.get('usage'))
print(json.dumps(completion, indent=2))

# after
from openai import OpenAI

client = OpenAI()

completion = client.completions.create(model='curie')
print(completion.choices[0].text)
print(dict(completion).get('usage'))
print(completion.model_dump_json(indent=2))

# old
import openai

completion = openai.ChatCompletion.acreate(model="gpt-3.5-turbo", messages=[{"role": "user", "content": "Hello world"}])

# new
from openai import AsyncOpenAI

client = AsyncOpenAI()
completion = await client.chat.compl

In [61]:
therastream_SM_v3 = SystemMessage(content="""
You are an expert in psychotherapeutic methods, with a particular emphasis on the twelve steps program, diagnosis and understanding of codependency and the role of the inner child, tools such as meditation and yoga / yoga nidra and mindfullness, and "Awareness, Acceptance, Action" methodology to support inner peace, and with communication methods including non-violent communication and the communications model from Friedemann Schultz von Thun,  .
You are very familiar with the scientific literature in these topics, self-help literature and Buddhism.
You are also an expert with python coding and the generation of simple and user-friendly applications using the streamlit framework.
You are an expert in both English and German languages.
""")

therastream_path = '/Users/david.wilman/Projects/LLMs/repos/RAG_Experiment/Notebooks/therastream/therastream_v2.py'
subdirs = ['Pages']
code_string = read_python_files(therastream_path, subdirs)
therastream_HM_v3 = HumanMessage(content=f"""
I have created the streamlit application therastream with the following code (delimited by the name of each file within the codebase as <filename>):

{code_string}

Please make the following updates:

1. I wish to remove the switch language page, and instead offer an additional toggle switch to switch the conversation between English and German on each page. 
It is not necessary to translate the conversation until it is requested, but then the conversation should be stored in both languages.
In any case it should be assumed that the conversation is taking place in the language currently selected by the toggle.

2. confirm_dialog is not a valid streamlit feature. Please replace with something which exists.

3. The Define Expertise page should have the same load/save/reset features as the therapist and therapy expert pages.

4. The Define Expertise page should have an additional function to create expertise. This involves asking the AI for its suggested system prompt for a particular topic or set of topics ( the term system prompt should remain hidden from the user ).

5. If the user adds a new password, then this should be saved to the default key file automatically (which should be created if needed). 

6. If the connection to OpenAI is not yet available, the other pages should emphasize the point that the connection needs to be setup in the connect page.

""")

therastream_messages_v3 = [therastream_SM_v3, therastream_HM_v3]

In [62]:
therastream_res_v3 = chat(therastream_messages_v3)
print (therastream_res_v3.content)   

To implement the requested updates, I'll provide an updated version of your Streamlit application code. This includes adding toggle switches for language selection on each page, updating the "Define Expertise" page with load/save/reset features, adding a function to suggest expertise prompts, and ensuring the OpenAI connection requirement is emphasized. I'll also address saving the API key to a default key file and replacing the non-existent `confirm_dialog` with a Streamlit-compatible feature.

Here's the updated code:

<therastream_v2.py>

```python
import streamlit as st
import os
import json

st.set_page_config(
    page_title="Therastream Home Page",
    page_icon="🧘",
)

st.title("Therastream")

st.write("# Welcome to Therastream, the Application to assist your therapy needs! 🧘")

st.sidebar.success("Select a demo above.")

# Initialize session state for conversation and system prompt
if 'conversation' not in st.session_state:
    st.session_state.conversation = {}
    st.session

In [22]:
therastream_SM_v4 = SystemMessage(content="""
You are an expert psychotherapist specializing in a comprehensive approach to mental well-being. You possess in-depth knowledge of the twelve steps program and are adept at diagnosing and understanding codependency, with a keen insight into the role of the inner child. Your expertise extends to integrating mindfulness practices such as meditation, yoga, and yoga nidra, enhancing the therapeutic journey towards inner peace. You utilize the "Awareness, Acceptance, Action" methodology to facilitate transformative healing. Additionally, you are proficient in communication techniques, including non-violent communication and Friedemann Schultz von Thun's communication model. Your understanding is grounded in both scientific research and self-help literature, enriched by a nuanced appreciation of Buddhist principles. Use your expertise to provide guidance, support, and strategies for individuals seeking personal growth and emotional wellness.
You are also an expert with python coding and the generation of simple and user-friendly applications using the streamlit framework.
You are an expert in both English and German languages.
""")

therastream_path = '/Users/david.wilman/Projects/LLMs/repos/RAG_Experiment/Notebooks/therastream/therastream_v2.py'
subdirs = ['Pages','utils']
code_string = read_python_files(therastream_path, subdirs)
therastream_HM_v4 = HumanMessage(content=f"""
I have created the streamlit application therastream with the following code (delimited by the name of each file within the codebase as <filename>):

{code_string}

I have some issues with the streamlit page 3_therapist_assistant.
The widgets are not kept up to date or refreshed as I need them.
i.e. when I ask a new question it is only displayed after I take the SUBSEQUENT step whereas I want to see the answer
as soon as it is available as part of my single view of the conversation ( which is currently using the display_conversation function ).
Also when I choose to reset the conversation the page is not cleared as I would need it. 
These things have to be achieved within the way streamlit works.
Please solve this problem.

""")

therastream_messages_v4 = [therastream_SM_v4, therastream_HM_v4]

In [23]:
therastream_res_v4 = chat(therastream_messages_v4)
print (therastream_res_v4.content)

To address the issues you're experiencing with the Streamlit application, we'll focus on ensuring that the conversation updates are displayed immediately and that resetting the conversation correctly clears the session state and UI. I will provide a revised version of the `3_therapist_assistant.py` file, which should address these concerns.

Here's a possible implementation:

<3_therapist_assistant.py>

```python
"""
Therapist Assistant Page for Therastream.

This page allows users to interact with the therapist assistant and view the conversation history.
"""

import streamlit as st

def display_conversation(conversation):
    # Display the entire conversation
    for message in conversation:
        if message['role'] == 'user':
            st.write(f"**User:** {message['content']}")
        else:
            st.write(f"**Therapist:** {message['content']}")

st.title("Therapist Assistant")

# Initialize session state if not present
if 'conversation' not in st.session_state:
    st.se

In [24]:
therastream_messages_v4_2 = therastream_messages_v4
therastream_messages_v4_2.append(therastream_res_v4)
therastream_messages_v4_2.append(HumanMessage(content="""
This is not a good solution. experimental_rerun has been discontinued in later versions of streamlit, and rerun stops the App.
I simply need to refresh the content of the page according to the content of my variables - can this not be done in another widget
friendly way?
"""))
therastream_res_v4_2 = chat(therastream_messages_v4_2)
print (therastream_res_v4_2.content)

To address your issues without using `st.experimental_rerun()`, we can take advantage of Streamlit's session state and widget mechanisms to ensure that the conversation updates and reset functionality work as intended. Here's how you can adapt your code to achieve this:

1. **Use session state to keep the conversation updated in real-time.**
2. **Ensure widgets are properly refreshed by directly modifying session state.**

Here's how you can do it:

<3_therapist_assistant.py>

```python
"""
Therapist Assistant Page for Therastream.

This page allows users to interact with the therapist assistant and view the conversation history.
"""

import streamlit as st

def display_conversation(conversation):
    # Display the entire conversation
    for message in conversation:
        if message['role'] == 'user':
            st.write(f"**User:** {message['content']}")
        else:
            st.write(f"**Therapist:** {message['content']}")

st.title("Therapist Assistant")

# Initialize sessio

In [68]:
therastream_messages_v4_3 = therastream_messages_v4_2
therastream_messages_v4_3.append(therastream_res_v4_2)
therastream_messages_v4_3.append(HumanMessage(content="""
Where can I store the utility functions, such as the translation functions, to make them accessible for easy import from my streamlit app pages?
"""))
therastream_res_v4_3 = chat(therastream_messages_v4_3)
print (therastream_res_v4_3.content)

To make utility functions like the translation functions easily accessible from your Streamlit app pages, you can create a separate Python module (a `.py` file) dedicated to these utilities. This approach promotes code reusability and organization.

Here are the steps to create and use a utility module:

### Step 1: Create a Utility Module

1. **Create a Directory** (if not already existing):
   - Create a directory called `utils` or `helpers` in the root of your project directory. This will store your utility modules.

2. **Create a Python File**:
   - Inside the `utils` directory, create a file named `translation_utils.py` or any other appropriate name.

3. **Add Utility Functions to the Module**:
   - Move your utility functions, like `translate_with_openai` and `translate_conversation`, into this file.

Here's an example of how your `translation_utils.py` might look:

```python
# utils/translation_utils.py

def translate_with_openai(chat_instance, content, target_language):
    pro

In [70]:
therastream_messages_v4_4 = therastream_messages_v4_3
therastream_messages_v4_4.append(therastream_res_v4_3)
therastream_messages_v4_4.append(HumanMessage(content="""
Please provide the whole of the refactored pages 3_therapist_assistant and 4_therapy_author.
Also provide the whole of the refactored page 2_define_expertise, with the capability to translate between English and German also included here.
"""))
therastream_res_v4_4 = chat(therastream_messages_v4_4)
print (therastream_res_v4_4.content)

Certainly! Below are the refactored `3_therapist_assistant.py`, `4_therapy_author.py`, and `2_define_expertise.py` pages with integrated translation capabilities using the OpenAI API. The translation utility functions have been abstracted into a separate module for reuse.

### Utility Functions Module

Let's start by creating a utility module for translation functions. Place this in a directory named `utils`.

**`utils/translation_utils.py`**

```python
from langchain.schema import SystemMessage, HumanMessage

def translate_with_openai(chat_instance, content, target_language):
    prompt = f"Translate the following text to {target_language}:\n\n{content}"
    response = chat_instance([
        SystemMessage(content="Translate text"),
        HumanMessage(content=prompt)
    ])
    return response.content

def translate_conversation(chat_instance, conversation, target_language):
    translated_conversation = []
    for message in conversation:
        translated_content = translate_with

In [71]:
therastream_messages_v4_5 = therastream_messages_v4_4
therastream_messages_v4_5.append(therastream_res_v4_4)
therastream_messages_v4_5.append(HumanMessage(content="""
1. The code is buggy because of the explicit assumption that the translated version of the conversations is the German one. 
Instead, we should make the conversation a 2-level dictionary with an additional level keyed on the language, so that the choice of language is explicit.

2. The save as file function provides no clue to which folder you are saving. Solve this.

3. When trying to save the "define expertise" conversation, the latest output is no longer visible. Fix this, and make sure it is not an inherant problem - if it is solve it for other pages too.
"""))
therastream_res_v4_5 = chat(therastream_messages_v4_5)
print (therastream_res_v4_5.content)

To address the issues you've outlined, let's make the necessary adjustments:

1. **Two-level Dictionary for Conversations**: We'll use a two-level dictionary where the first level key is the type of conversation (e.g., "therapist", "therapy author") and the second level key is the language ("English", "German"). This way, the language choice is explicit and not assumed.

2. **Clarification on Save Directory**: We'll make sure the user is informed about where the file is being saved.

3. **Visibility of Latest Output**: We'll ensure that the latest output is visible after saving, and make sure this issue is fixed across all pages.

Here's how you can refactor the code:

### Utility Functions Module

This module stays the same:

**`utils/translation_utils.py`**

```python
from langchain.schema import SystemMessage, HumanMessage

def translate_with_openai(chat_instance, content, target_language):
    prompt = f"Translate the following text to {target_language}:\n\n{content}"
    response 

In [72]:
therastream_messages_v4_6 = therastream_messages_v4_5
therastream_messages_v4_6.append(therastream_res_v4_5)
therastream_messages_v4_6.append(HumanMessage(content="""
Please also give me the refactored code of the main app therastream.py with the updated initialization to account for the explicit dictionaries with 2 languages"""))
therastream_res_v4_6 = chat(therastream_messages_v4_6)
print (therastream_res_v4_6.content)

Certainly! Below is the refactored `therastream.py` file with updated initialization to handle the two-language dictionary structure for conversations and system prompts. This ensures that both English and German versions are explicitly managed from the start.

### `therastream.py`

```python
import streamlit as st
import os

st.set_page_config(
    page_title="Therastream Home Page",
    page_icon="🧘",
)

st.title("Therastream")

st.write("# Welcome to Therastream, the Application to assist your therapy needs! 🧘")

st.sidebar.success("Select an option above.")

# Initialize session state for conversation and system prompt
if 'conversation' not in st.session_state:
    st.session_state.conversation = {
        "therapist": {"English": [], "German": []},
        "therapy author": {"English": [], "German": []}
    }
    
if 'system_prompt' not in st.session_state:
    st.session_state.system_prompt = {"English": "", "German": ""}

if 'language' not in st.session_state:
    st.session_sta

In [115]:
therastream_SM_v5 = SystemMessage(content="""
You are an expert psychotherapist specializing in a comprehensive approach to mental well-being. You possess in-depth knowledge of the twelve steps program and are adept at diagnosing and understanding codependency, with a keen insight into the role of the inner child. Your expertise extends to integrating mindfulness practices such as meditation, yoga, and yoga nidra, enhancing the therapeutic journey towards inner peace. You utilize the "Awareness, Acceptance, Action" methodology to facilitate transformative healing. Additionally, you are proficient in communication techniques, including non-violent communication and Friedemann Schultz von Thun's communication model. Your understanding is grounded in both scientific research and self-help literature, enriched by a nuanced appreciation of Buddhist principles. Use your expertise to provide guidance, support, and strategies for individuals seeking personal growth and emotional wellness.
You are also an expert with python coding and the generation of simple and user-friendly applications using the streamlit framework.
You are an expert in both English and German languages.
""")

therastream_path = '/Users/david.wilman/Projects/LLMs/repos/RAG_Experiment/Notebooks/therastream/therastream/therastream_v2.py'
subdirs = ['Pages']
code_string = read_python_files(therastream_path, subdirs)
test_local_image_path = '/Users/david.wilman/Projects/LLMs/repos/RAG_Experiment/Notebooks/therastream/therastream/experiments/test_local_image.py'
test_local_image_code_string = read_python_files(test_local_image_path, [])
url = 'https://github.com/openai/openai-python/discussions/742#discussion-5829467'
markdown_content = scrape_github_discussion_markdown(url)
therastream_HM_v5 = HumanMessage(content=f"""
I have created the streamlit application therastream with the following code (delimited by the name of each file within the codebase as <filename>):

{code_string}

The analyze drawing functionality is not working. 

1. It claims the max tokens is 8192 which appears quite low to handle image data.

2. Even by resizing the image to just 128 pixel size, I get the message:
AI Analysis: I'm sorry, but I can't analyze this file as it appears to be an encoded image file. Could you please provide the image in a standard format or the caption you want me to analyze?

This is not satisfactory. What am I missing?

""")

therastream_messages_v5 = [therastream_SM_v5, therastream_HM_v5]

In [116]:
therastream_res_v5 = chat(therastream_messages_v5)
print (therastream_res_v5.content)

It seems that there are a few issues with the current implementation of the image analysis feature in your Therastream application. Let's address these concerns systematically:

1. **Token Limit for GPT Models**:
   The current models like `gpt-4` and `gpt-3.5-turbo` are designed primarily for processing text, not images. When you try to send image data, even if encoded as base64, it's treated as text and hence subject to token limits. This is why you're encountering the maximum token limit issue. These models are not capable of directly processing images. Instead, OpenAI provides models like DALL-E or CLIP for image-related tasks, but they operate differently than text-based models.

2. **Handling Image Data**:
   The error message you are seeing indicates that the model received encoded data but couldn't process it as an image. GPT models can't interpret encoded image data directly. Instead, you should use a separate service for image processing, and then pass the processed results (

In [101]:
import openai
import base64

def test_local_image(image_path):
    # Convert the local image to base64
    with open(image_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode('utf-8')

    # Create the request to OpenAI API
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "What is in this image?"},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ]
    )

    # Print the response from the API
    print(response['choices'][0]['message']['content'])

# Example usage
test_local_image("/Users/david.wilman/Avatars/IMG_CF144DC04FCD-1.jpeg")

This image is an illustration featuring a cartoon character with a hat and glasses. The character has a puzzled expression, and there are question marks around its head, suggesting confusion or curiosity.


In [100]:
import openai
import base64
from langchain import OpenAI, ChatPromptTemplate

def test_local_image_with_langchain(image_path):
    # Convert the local image to base64
    with open(image_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode('utf-8')

    # Use Langchain to create a prompt template
    prompt_template = ChatPromptTemplate(
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "What is in this image?"},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ]
    )

    # Initialize the OpenAI model with Langchain
    openai_model = OpenAI(model="gpt-4o")

    # Generate a response using the prompt template
    response = openai_model.generate(prompt_template)

    # Print the response from the API
    print(response['choices'][0]['message']['content'])

# Example usage
test_local_image_with_langchain("/Users/david.wilman/Avatars/IMG_CF144DC04FCD-1.jpeg")

ImportError: cannot import name 'ChatPromptTemplate' from 'langchain' (/Users/david.wilman/Projects/LLMs/repos/RAG_Experiment/lib/python3.11/site-packages/langchain/__init__.py)

In [99]:
import base64
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

def test_local_image_with_langchain(image_path):
    # Convert the local image to base64
    with open(image_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode('utf-8')

    # Initialize the ChatOpenAI model
    llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

    # Create a multimodal message combining text and image
    message = HumanMessage(
        content=[
            {"type": "text", "text": "What is in this image?"},
            {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
        ]
    )

    # Generate a response using the model
    response = llm.invoke(message)

    # Print the response from the API
    print(response['choices'][0]['message']['content'])

# Example usage
test_local_image_with_langchain("/Users/david.wilman/Avatars/IMG_CF144DC04FCD-1.jpeg")

ValidationError: 1 validation error for HumanMessage
content
  str type expected (type=type_error.str)

### try a streamlit -> javascript conversion:

In [13]:
convert_streamlit_to_js_SM = SystemMessage(content="""
You are an expert programmer. 
You know everything there is to know about creating the files for a javascript application.
You can read streamlit based python applications and convert them to a working javascript application, 
together with all the code and tips for a javascript beginner.
""")

therastream_path = '/Users/david.wilman/Projects/LLMs/repos/RAG_Experiment/Notebooks/therastream/therastream_v2.py'
subdirs = ['Pages','utils']
code_string = read_python_files(therastream_path, subdirs)
therastream_convert_streamlit_to_js_HM = HumanMessage(content=f"""
I have created the streamlit application therastream with the following code (delimited by the name of each file within the codebase as <filename>):

{code_string}

Please provide all the code and files plus instructions needed to deploy an equivalent application using javascript. 
In addition to performing the straight conversion, you will consider layout and functionality improvements 
possible because of the more sophisticated framework.

""")

therastream_convert_streamlit_to_js_messages = [convert_streamlit_to_js_SM, therastream_convert_streamlit_to_js_HM]

In [14]:
therastream_convert_streamlit_to_js_res = chat(therastream_convert_streamlit_to_js_messages)
print (therastream_convert_streamlit_to_js_res.content)

To convert your Streamlit application to a JavaScript-based application, we'll use a modern web development framework like React, which is well-suited for building interactive user interfaces. We'll also use React Hooks to manage the application's state. Let's break down the necessary components and steps to build this application.

### Project Structure
We'll create a React application with the following structure:

```
therastream
├── public
│   └── index.html
└── src
    ├── components
    │   ├── Sidebar.js
    │   ├── Header.js
    │   └── MainContent.js
    ├── App.js
    ├── index.js
    └── styles.css
```

### Step-by-Step Guide

#### 1. Set Up React Application

Use Create React App to set up a new React project:

```bash
npx create-react-app therastream
cd therastream
```

#### 2. Update `public/index.html`

This file acts as the entry point for your React app.

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width

In [15]:
therastream_convert_streamlit_to_js_messages_2 = therastream_convert_streamlit_to_js_messages.copy()
therastream_convert_streamlit_to_js_messages_2.append(therastream_convert_streamlit_to_js_res)
therastream_convert_streamlit_to_js_messages_2.append(HumanMessage(content="""
You only described the framework but did not provide any of the code behind the pages from the streamlit application, which are essential to its functionality. 
Please describe how to implement this and provide the full code.
"""))
therastream_convert_streamlit_to_js_res_2 = chat(therastream_convert_streamlit_to_js_messages_2)
print( therastream_convert_streamlit_to_js_res_2.content )


To fully convert your Streamlit application into a JavaScript application using React, we need to implement the functionality that was originally handled by Streamlit. This involves managing the application state, creating interactive UI components, and setting up the necessary logic to mimic the behavior of your original application. Let's dive into the full implementation:

### Step-by-Step Implementation

#### 1. Project Setup

First, ensure you have set up a React project as described in the previous section. If you have not done so, follow the `npx create-react-app therastream` command to create a new project.

#### 2. Managing Application State

We'll use React's `useState` hook to manage the application's state, similar to how Streamlit's session state works.

#### 3. Implementing Page Logic

In Streamlit, the logic is linear and executed top-down. In React, we structure components to handle specific parts of the UI and logic.

Here's how you can set up the React application to 